In [19]:
from flair.data import TaggedCorpus, Sentence
from flair.data_fetcher import NLPTaskDataFetcher
import json
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, FlairEmbeddings, ELMoEmbeddings
from typing import List
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

from hyperopt import hp
from flair.hyperparameter.param_selection import SearchSpace, Parameter, SEQUENCE_TAGGER_PARAMETERS
from flair.hyperparameter import SequenceTaggerParamSelector
from flair.hyperparameter.param_selection import OptimizationValue

from pathlib import Path

In [5]:
import emoji

In [6]:
columns = {0: 'text', 1: 'pos'}

In [7]:
data_folder = "../data"

In [8]:
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(
    data_folder, columns,
    train_file='train.txt',
    dev_file='dev.txt',
    test_file='test.txt'
)

2019-05-13 22:18:04,632 Reading data from ../data
2019-05-13 22:18:04,635 Train: ../data/train.txt
2019-05-13 22:18:04,636 Dev: ../data/dev.txt
2019-05-13 22:18:04,638 Test: ../data/test.txt


In [23]:
corpus = corpus.downsample(0.1)

In [24]:
tag_dict = corpus.make_tag_dictionary(tag_type='pos')

In [25]:
len(corpus.train), len(corpus.dev), len(corpus.test)

(7437, 1860, 2324)

In [12]:
len(tag_dict.idx2item)

53

In [14]:
SEQUENCE_TAGGER_PARAMETERS

['embeddings',
 'hidden_size',
 'rnn_layers',
 'use_crf',
 'use_rnn',
 'dropout',
 'locked_dropout',
 'word_dropout']

In [32]:
search_space = SearchSpace()
search_space.add(Parameter.EMBEDDINGS, hp.choice, options=[
    FlairEmbeddings("../models/flair-lm/best-lm.pt"),
    StackedEmbeddings([WordEmbeddings('twitter'), FlairEmbeddings('../models/flair-lm/best-lm.pt')]),
    ELMoEmbeddings(),
    WordEmbeddings('twitter'), 
    StackedEmbeddings([FlairEmbeddings('news-forward'), FlairEmbeddings('news-backward')]),
    StackedEmbeddings([WordEmbeddings('twitter'), FlairEmbeddings('news-forward'), FlairEmbeddings('news-backward')]),
    StackedEmbeddings([ELMoEmbeddings(), WordEmbeddings('twitter')])
])
search_space.add(Parameter.HIDDEN_SIZE, hp.choice, options=[32, 64, 128, 256])
search_space.add(Parameter.RNN_LAYERS, hp.choice, options=[1, 2])
search_space.add(Parameter.DROPOUT, hp.uniform, low=0.0, high=0.5)
search_space.add(Parameter.LEARNING_RATE, hp.choice, options=[0.05, 0.1, 0.15, 0.2])
search_space.add(Parameter.MINI_BATCH_SIZE, hp.choice, options=[8, 16, 32])

2019-05-13 22:35:57,579 this function is deprecated, use smart_open.open instead
2019-05-13 22:36:13,239 this function is deprecated, use smart_open.open instead
2019-05-13 22:36:20,294 this function is deprecated, use smart_open.open instead
2019-05-13 22:36:36,408 this function is deprecated, use smart_open.open instead


In [33]:
search_space.add(Parameter.USE_CRF, hp.choice, options=[True, False])

In [34]:
param_selector = SequenceTaggerParamSelector(
    corpus, 'pos', '../models/flair-hyperoptim',
    max_epochs=10, evaluation_metric=EvaluationMetric.MICRO_F1_SCORE,
    training_runs=3, optimization_value=OptimizationValue.DEV_SCORE
)

In [ ]:
param_selector.optimize(search_space, max_evals=100)

  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]2019-05-13 22:37:10,291 ----------------------------------------------------------------------------------------------------
2019-05-13 22:37:10,292 Evaluation run: 1
2019-05-13 22:37:10,293 Evaluating parameter combination:
2019-05-13 22:37:10,294 	dropout: 0.2973741038304056
2019-05-13 22:37:10,295 	embeddings: elmo-original
2019-05-13 22:37:10,296 	hidden_size: 64
2019-05-13 22:37:10,297 	learning_rate: 0.2
2019-05-13 22:37:10,298 	mini_batch_size: 32
2019-05-13 22:37:10,298 	rnn_layers: 1
2019-05-13 22:37:10,299 	use_crf: True
2019-05-13 22:37:10,300 ----------------------------------------------------------------------------------------------------
2019-05-13 22:37:10,386 ----------------------------------------------------------------------------------------------------
2019-05-13 22:37:10,387 Training run: 1
2019-05-13 22:37:10,464 ------------------------------------------------------------------------------------------------

2019-05-13 22:41:15,396 epoch 7 - iter 138/233 - loss 5.27520728
2019-05-13 22:41:18,240 epoch 7 - iter 161/233 - loss 5.30657030
2019-05-13 22:41:20,994 epoch 7 - iter 184/233 - loss 5.31360569
2019-05-13 22:41:23,878 epoch 7 - iter 207/233 - loss 5.31241672
2019-05-13 22:41:26,669 epoch 7 - iter 230/233 - loss 5.28373152
2019-05-13 22:41:26,859 ----------------------------------------------------------------------------------------------------
2019-05-13 22:41:26,861 EPOCH 7 done: loss 5.2849 - lr 0.2000 - bad epochs 0
2019-05-13 22:41:32,984 DEV  : loss 4.78375959 - f-score 0.2210 - acc 0.1242
2019-05-13 22:41:32,986 ----------------------------------------------------------------------------------------------------
2019-05-13 22:41:33,124 epoch 8 - iter 0/233 - loss 4.40708160
2019-05-13 22:41:36,080 epoch 8 - iter 23/233 - loss 5.45272603
2019-05-13 22:41:38,921 epoch 8 - iter 46/233 - loss 5.25193337
2019-05-13 22:41:41,733 epoch 8 - iter 69/233 - loss 5.24632652
2019-05-13 22:41

2019-05-13 22:43:29,396 :pleading_face: tp: 0 - fp: 0 - fn: 19 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:43:29,397 :purple_heart: tp: 0 - fp: 0 - fn: 51 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:43:29,397 :raising_hands: tp: 0 - fp: 0 - fn: 49 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:43:29,398 :red_heart: tp: 93 - fp: 48 - fn: 117 - tn: 93 - precision: 0.6596 - recall: 0.4429 - accuracy: 0.3605 - f1-score: 0.5300
2019-05-13 22:43:29,399 :relieved_face: tp: 0 - fp: 0 - fn: 28 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:43:29,399 :rolling_on_the_floor_laughing: tp: 0 - fp: 0 - fn: 111 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:43:29,400 :skull:    tp: 0 - fp: 0 - fn: 38 - tn: 0 - precision: 0.0000 - recall: 0.0000 - acc

2019-05-13 22:45:48,675 epoch 5 - iter 0/233 - loss 6.08601284
2019-05-13 22:45:51,534 epoch 5 - iter 23/233 - loss 5.19152687
2019-05-13 22:45:54,446 epoch 5 - iter 46/233 - loss 5.58141479
2019-05-13 22:45:57,195 epoch 5 - iter 69/233 - loss 5.53912959
2019-05-13 22:46:00,083 epoch 5 - iter 92/233 - loss 5.47954253
2019-05-13 22:46:02,907 epoch 5 - iter 115/233 - loss 5.45790356
2019-05-13 22:46:05,769 epoch 5 - iter 138/233 - loss 5.50058934
2019-05-13 22:46:08,490 epoch 5 - iter 161/233 - loss 5.48315895
2019-05-13 22:46:11,202 epoch 5 - iter 184/233 - loss 5.45004545
2019-05-13 22:46:13,976 epoch 5 - iter 207/233 - loss 5.43493311
2019-05-13 22:46:16,802 epoch 5 - iter 230/233 - loss 5.43087754
2019-05-13 22:46:17,021 ----------------------------------------------------------------------------------------------------
2019-05-13 22:46:17,023 EPOCH 5 done: loss 5.4374 - lr 0.2000 - bad epochs 0
2019-05-13 22:46:23,172 DEV  : loss 4.92100096 - f-score 0.1819 - acc 0.1000
2019-05-13 2

2019-05-13 22:49:23,241 :face_blowing_a_kiss: tp: 0 - fp: 0 - fn: 21 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:49:23,242 :face_with_rolling_eyes: tp: 0 - fp: 0 - fn: 34 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:49:23,243 :face_with_tears_of_joy: tp: 236 - fp: 738 - fn: 231 - tn: 236 - precision: 0.2423 - recall: 0.5054 - accuracy: 0.1959 - f1-score: 0.3276
2019-05-13 22:49:23,243 :female_sign: tp: 90 - fp: 93 - fn: 2 - tn: 90 - precision: 0.4918 - recall: 0.9783 - accuracy: 0.4865 - f1-score: 0.6546
2019-05-13 22:49:23,244 :fire:     tp: 0 - fp: 0 - fn: 79 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:49:23,244 :flexed_biceps: tp: 0 - fp: 0 - fn: 39 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 22:49:23,245 :flushed_face: tp: 0 - fp: 0 - fn: 41 - tn: 0 - precision: 0.0000 - recall

...

2019-05-13 23:07:04,968 Testing using best model ...
2019-05-13 23:07:12,619 MICRO_AVG: acc 0.1344 - f1-score 0.2369
2019-05-13 23:07:12,621 MACRO_AVG: acc 0.0397 - f1-score 0.06270816326530612
2019-05-13 23:07:12,623 :OK_hand:  tp: 0 - fp: 0 - fn: 21 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:07:12,624 :backhand_index_pointing_down: tp: 0 - fp: 0 - fn: 16 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:07:12,625 :beaming_face_with_smiling_eyes: tp: 0 - fp: 0 - fn: 26 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:07:12,626 :black_heart: tp: 15 - fp: 0 - fn: 16 - tn: 15 - precision: 1.0000 - recall: 0.4839 - accuracy: 0.4839 - f1-score: 0.6522
2019-05-13 23:07:12,627 :blue_heart: tp: 0 - fp: 0 - fn: 35 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:07:12,628 :clapping_hands: tp: 0 - fp:

2019-05-13 23:07:18,542 epoch 1 - iter 46/233 - loss 10.52431306
2019-05-13 23:07:21,451 epoch 1 - iter 69/233 - loss 9.40511389
2019-05-13 23:07:24,307 epoch 1 - iter 92/233 - loss 8.76840318
2019-05-13 23:07:27,262 epoch 1 - iter 115/233 - loss 8.33754576
2019-05-13 23:07:30,212 epoch 1 - iter 138/233 - loss 8.02513294
2019-05-13 23:07:33,099 epoch 1 - iter 161/233 - loss 7.79891802
2019-05-13 23:07:35,823 epoch 1 - iter 184/233 - loss 7.60171776
2019-05-13 23:07:38,678 epoch 1 - iter 207/233 - loss 7.42066569
2019-05-13 23:07:41,395 epoch 1 - iter 230/233 - loss 7.25755067
2019-05-13 23:07:41,623 ----------------------------------------------------------------------------------------------------
2019-05-13 23:07:41,625 EPOCH 1 done: loss 7.2504 - lr 0.1500 - bad epochs 0
2019-05-13 23:07:47,733 DEV  : loss 5.44599819 - f-score 0.0832 - acc 0.0434
2019-05-13 23:07:47,735 ----------------------------------------------------------------------------------------------------
2019-05-13 23

2019-05-13 23:11:43,580 epoch 8 - iter 207/233 - loss 4.78648124
2019-05-13 23:11:46,391 epoch 8 - iter 230/233 - loss 4.77463772
2019-05-13 23:11:46,633 ----------------------------------------------------------------------------------------------------
2019-05-13 23:11:46,635 EPOCH 8 done: loss 4.7742 - lr 0.1500 - bad epochs 0
2019-05-13 23:11:52,790 DEV  : loss 4.51527596 - f-score 0.2484 - acc 0.1418
2019-05-13 23:11:52,793 ----------------------------------------------------------------------------------------------------
2019-05-13 23:11:52,928 epoch 9 - iter 0/233 - loss 5.02456093
2019-05-13 23:11:55,831 epoch 9 - iter 23/233 - loss 4.53863839
2019-05-13 23:11:58,573 epoch 9 - iter 46/233 - loss 4.57298913
2019-05-13 23:12:01,369 epoch 9 - iter 69/233 - loss 4.61002951
2019-05-13 23:12:04,050 epoch 9 - iter 92/233 - loss 4.71773321
2019-05-13 23:12:06,902 epoch 9 - iter 115/233 - loss 4.77332186
2019-05-13 23:12:09,591 epoch 9 - iter 138/233 - loss 4.73370125
2019-05-13 23:12:

2019-05-13 23:13:12,085 :smiling_face_with_smiling_eyes: tp: 0 - fp: 0 - fn: 34 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:13:12,085 :smiling_face_with_sunglasses: tp: 0 - fp: 0 - fn: 23 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:13:12,086 :smirking_face: tp: 0 - fp: 0 - fn: 22 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:13:12,087 :sparkles: tp: 0 - fp: 0 - fn: 38 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:13:12,087 :sparkling_heart: tp: 0 - fp: 0 - fn: 19 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:13:12,087 :speaking_head: tp: 0 - fp: 0 - fn: 14 - tn: 0 - precision: 0.0000 - recall: 0.0000 - accuracy: 0.0000 - f1-score: 0.0000
2019-05-13 23:13:12,088 :thinking_face: tp: 12 - fp: 8 - fn: 43 - tn: 12 - precision: 0.6000 - 

2019-05-13 23:15:24,794 EPOCH 4 done: loss 5.3753 - lr 0.0500 - bad epochs 0
2019-05-13 23:15:30,855 DEV  : loss 4.90589046 - f-score 0.2000 - acc 0.1111
2019-05-13 23:15:30,858 ----------------------------------------------------------------------------------------------------
2019-05-13 23:15:30,985 epoch 5 - iter 0/233 - loss 3.82064295
2019-05-13 23:15:33,584 epoch 5 - iter 23/233 - loss 5.07902658
2019-05-13 23:15:36,192 epoch 5 - iter 46/233 - loss 5.14742368
2019-05-13 23:15:38,969 epoch 5 - iter 69/233 - loss 5.20373874
2019-05-13 23:15:41,696 epoch 5 - iter 92/233 - loss 5.24345217
2019-05-13 23:15:44,455 epoch 5 - iter 115/233 - loss 5.26348478
2019-05-13 23:15:47,196 epoch 5 - iter 138/233 - loss 5.25827106
2019-05-13 23:15:49,929 epoch 5 - iter 161/233 - loss 5.24821651
2019-05-13 23:15:52,644 epoch 5 - iter 184/233 - loss 5.24921899
2019-05-13 23:15:55,165 epoch 5 - iter 207/233 - loss 5.24356434
2019-05-13 23:15:57,881 epoch 5 - iter 230/233 - loss 5.24334452
2019-05-13 2